In [1]:
import pandas as pd
import numpy as np
import os

from folium.features import DivIcon
import folium
import pyproj
from pyproj import Proj, transform

%matplotlib inline  
import matplotlib.pyplot as plt 
import seaborn as sns 

import warnings
warnings.filterwarnings(action='ignore')

def katec_to_wgs84(x, y):
    inProj  = Proj('+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43')
    outProj = Proj({ 'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84' })
    return transform( inProj, outProj, x, y )

WGS84 = { 'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84', }
KATEC = { 'proj':'tmerc', 'lat_0':'38N', 'lon_0':'128E', 'ellps':'bessel','x_0':'400000', 'y_0':'600000', 'k':'0.9999','towgs84':'-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43'}
def wgs84_to_katec(longitude, latitude):
    return transform( Proj(**WGS84), Proj(**KATEC), longitude, latitude )

In [7]:
pwd

'C:\\Users\\dudtj\\busan_moving_person\\data\\parquet_data'

In [6]:
cd data/parquet_data/

[WinError 3] 지정된 경로를 찾을 수 없습니다: 'data/parquet_data/'
C:\Users\dudtj\busan_moving_person\data\parquet_data


In [154]:
kcelldf = pd.read_parquet('50CELL_BUSANJINGU_2019_201912.parquet')
# kcelldf.head()

In [155]:
tdf = pd.DataFrame(kcelldf['total'].groupby(kcelldf['id']).sum()).reset_index()
geodf = pd.merge(tdf, kcelldf[['id', 'x', 'y']] , how = 'left', on = 'id')[['id', 'x', 'y', 'total']].drop_duplicates()

In [156]:
geodf

,id,x,y,total
0,95139666,492452,284793,6182.78
744,95151810,492502,282543,2797.52
1488,95151811,492502,282593,2797.52
2232,95151812,492502,282643,2797.52
2976,95151854,492502,284743,6182.78
...,...,...,...,...
5919210,96675484,498752,284993,9.77
5919219,96675511,498752,286343,1704.96
5919818,96675512,498752,286393,2761.27
5920558,96687671,498802,284893,2696.86


In [157]:
geodf.to_csv('201912_busan_dense.csv', index=False)

In [158]:
geodf['total'].describe()

count      9880.000000
mean       8736.398596
std       18648.012677
min           1.000000
25%        1212.707500
50%        3939.690000
75%        8979.807500
max      563194.950000
Name: total, dtype: float64

In [159]:
geodf[geodf['total'] > 8979.807500].describe()

,id,x,y,total
count,2.470000e+03,2470.000000,2470.000000,2470.000000
mean,9.592762e+07,495684.226721,284971.481781,25813.804166
std,3.730191e+05,1530.117485,1113.899955,31333.956016
min,9.521280e+07,492752.000000,282843.000000,8987.000000
25%,9.561503e+07,494402.000000,284093.000000,11441.050000
50%,9.600513e+07,496002.000000,284793.000000,15858.830000
75%,9.623668e+07,496952.000000,285880.500000,26914.367500
max,9.665113e+07,498652.000000,287743.000000,563194.950000


In [160]:
def f_color(x):
    if x > 26914.367500:        # 기존 75%를 넘는 범위에서의 4사분위수 
        a = '#FF4F00'        
    elif x > 8979.807500:      # 75%를 기준으로 설정
        a = '#FCB100'   
    elif x > 3939.690000:      # 50%를 기준으로 설정
        a = '#E0F500' 
    elif x > 1212.707500:      # 25%를 기준으로 설정
        a = '#8CF700'  
    else:
        a = '#00F000'
    return a

geodf['cell_color']= geodf['total'].apply(lambda x : f_color(x))

In [161]:
cell_size = 55
a = cell_size//2
x = geodf
x['nwx'], x['nwy'] = x['x']-a, x['y']+a
x['nex'], x['ney'] = x['x']+a, x['y']+a
x['swx'], x['swy'] = x['x']-a, x['y']-a
x['sex'], x['sey'] = x['x']+a ,x['y']-a

x['lng'], x['lat'] = katec_to_wgs84(x.x.to_list(), x.y.to_list())
x['nwlng'], x['nwlat'] = katec_to_wgs84(x.nwx.to_list(), x.nwy.to_list())
x['nelng'], x['nelat'] = katec_to_wgs84(x.nex.to_list(), x.ney.to_list())
x['swlng'], x['swlat'] = katec_to_wgs84(x.swx.to_list(), x.swy.to_list())
x['selng'], x['selat'] = katec_to_wgs84(x.sex.to_list(), x.sey.to_list())

In [162]:
map = folium.Map(location=[geodf['lat'].mean(), geodf['lng'].mean()], zoom_start=13)#, tiles='stamentoner')

for n in x.index:
    nw, ne, sw, se = [x.loc[n, 'nwlat'],x.loc[n, 'nwlng']],[x.loc[n, 'nelat'],x.loc[n, 'nelng']],[x.loc[n, 'swlat'],x.loc[n, 'swlng']],[x.loc[n, 'selat'],x.loc[n, 'selng']]
    cell_color = x.loc[n, 'cell_color']
    loc_name = 'ID:'+str(round(x.loc[n, 'id'],3)) + '       Total:'+ str(round(x.loc[n, 'total'],3))
    folium.Polygon(locations=[nw, sw, se, ne],
                   color='white',
                   popup=loc_name,
                   weight=0.1,
                   fill=True,
                   fill_color=cell_color,
                   fill_opacity=0.6
     ).add_to(map)

map.save('201912_busan_dense.html')
# map
print("done :)")

done :)
